# Necessary Packages

In [23]:
import PyPDF2 as pdf
import pandas as pd
import numpy as np
import math
import statistics
import operator
import matplotlib.pyplot as plt
import traitlets
from IPython.display import display
from ipywidgets import widgets
from tkinter import Tk, filedialog
from tokenize import tokenize
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import io
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import win32com.client
import codecs
from bs4 import BeautifulSoup
import textract
import docx
import os
import re
from tkinter import Tk, filedialog
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\simon\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
stop_words = frozenset(stopwords.words('english'))

# Pick File where syllabai are located 

In [3]:
root = Tk()
root.directory = filedialog.askdirectory()
directory = root.directory
root.withdraw() #hide the main window 

''

In [9]:
# initialize a list to add parsed text to
text_list = []

# Loop through the files in the directory
for filename in os.listdir(directory)[:50]: #only did 500 becuase it takes forever
    first_file = filename
    path = directory + '/' + first_file
    
    #handle parsing for different file types
    if '.pdf' in first_file:
        text = convert_pdf_to_txt_2(path)
    elif '.doc' in first_file:
        text = doc_parser(path)
    else:
        text = htm_parser(path)
    text_list.append(text)

### Initial Processing of Data to create TF-IDF

In [207]:
cv=CountVectorizer(max_df=0.85,stop_words=stop_words)
word_count_vector=cv.fit_transform(text_list)

In [208]:
list(cv.vocabulary_.keys())[:10]

['african',
 'middle',
 'eastern',
 'languages',
 'literatures',
 'language',
 'center',
 'university',
 'amelang',
 'beginning']

In [209]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

### Testing the keyword generation for document 

In [212]:
feature_names=cv.get_feature_names()
number = np.random.randint(0, 500)

In [222]:
count = 0
test_list = []
for filename in os.listdir(directory):
    if count == number:
        test_file = filename
        path = directory + '/' + test_file
        if '.pdf' in test_file:
            text = convert_pdf_to_txt_2(path)
        elif '.doc' in test_file:
            text = doc_parser(path)
        else:
            text = htm_parser(path)
        test_list.append(text)
    count+= 1

In [227]:
feature_names=cv.get_feature_names()
 
# get the document that we want to extract keywords from
doc=test_list[0]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)
 
# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])
 


=====Doc=====
 mark twain and the assault of laughter stanford university autumn english t mw hilton obenzinger office hours m w pm pm and by appointment office tba office phone tba home phone email obenzinger stanford edu for your human race in its poverty has unquestionably one really effective weapon laughter power money persuasion supplication persecution these can lift at a colossal humbug push it a little weaken it a little century by century but only laughter can blow it to rags and atoms at a blast against the assault of laughter nothing can stand from the mysterious stranger manuscripts description mark twain continues to delight and to disturb in this course we will examine how an outrageous phunny phellow notorious liar and irreverent blasphemer became a moral barometer the abraham lincoln of our literature against the assault of laughter nothing can stand twain wrote and his fictions satires and burlesques provided a constant comic barrage against the pretensions of his da

### Necessary Functions Defined below 

In [5]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [6]:
def convert_pdf_to_txt_2(path):
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos = set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,
                                  password=password,
                                  caching=caching,
                                  check_extractable=True):
        interpreter.process_page(page)
        
    fp.close()
    device.close()
    text = retstr.getvalue()
    text_vec = text.split('\n')
    text_total =  ' '
    for word in text_vec:
        text_total += ' ' + word
    retstr.close()
    
     # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    text_total=re.sub("(\\d|\\W)+"," ",text_total)
    
    return text_total

In [7]:
def htm_parser(path):
    f=codecs.open(path, 'r', encoding='latin-1')
    unsoup = f.read()
    soup = BeautifulSoup(unsoup)
    for script in soup(["script", "style"]):
        script.decompose()
    strips = list(soup.stripped_strings)
    text_total= ''
    for strip in strips:
        if (strip != 'Ê') and('Ê' not in strip):
            text_total += ' ' + strip.strip('\n')
     
    # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    text_total=re.sub("(\\d|\\W)+"," ",text_total)
    
    return text_total

In [8]:
def doc_parser(path):
    doc = win32com.client.GetObject(path)
    text= doc.Range().Text.split('\r')
    text_total = ' '
    for word in text:
        if '' in word:
            word.replace('', '')
        if word != '':
            text_total += ' ' + word
            
     # lowercase
    text_total=text_total.lower()
    
    #remove tags
    text_total=re.sub("<!--?.*?-->","",text_total)
    
    # remove special characters and digits
    text_total=re.sub("(\\d|\\W)+"," ",text_total)
    
    return text_total

In [16]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(text_list))

print(data_words[:1])

[['african', 'middle', 'eastern', 'languages', 'literatures', 'the', 'stanford', 'language', 'center', 'stanford', 'university', 'amelang', 'beginning', 'swahili', 'course', 'outline', 'fall', 'instructor', 'sangai', 'mohochi', 'office', 'building', 'room', 'phone', 'fax', 'office', 'hours', 'mt', 'pm', 'also', 'by', 'appointment', 'mail', 'mohochi', 'stanford', 'edu', 'introduction', 'this', 'is', 'the', 'first', 'part', 'of', 'three', 'quarter', 'beginning', 'swahili', 'course', 'the', 'purpose', 'of', 'the', 'course', 'is', 'to', 'introduce', 'the', 'student', 'to', 'swahili', 'language', 'and', 'equip', 'him', 'her', 'with', 'the', 'necessary', 'skills', 'to', 'be', 'able', 'to', 'read', 'write', 'speak', 'and', 'understand', 'basic', 'swahili', 'swahili', 'is', 'the', 'national', 'language', 'of', 'kenya', 'and', 'tanzania', 'and', 'uganda', 'to', 'lesser', 'extent', 'it', 'is', 'the', 'first', 'second', 'or', 'main', 'language', 'of', 'most', 'people', 'in', 'east', 'central', 'a

In [19]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=10) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=10)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['african', 'middle', 'eastern', 'languages', 'literatures', 'the', 'stanford', 'language_center', 'stanford_university', 'amelang', 'beginning', 'swahili', 'course', 'outline', 'fall', 'instructor', 'sangai', 'mohochi', 'office_building', 'room', 'phone', 'fax', 'office_hours', 'mt', 'pm', 'also', 'by_appointment', 'mail', 'mohochi', 'stanford_edu', 'introduction', 'this', 'is', 'the', 'first', 'part', 'of', 'three', 'quarter', 'beginning', 'swahili', 'course', 'the', 'purpose', 'of', 'the', 'course', 'is', 'to', 'introduce', 'the', 'student', 'to', 'swahili', 'language', 'and', 'equip', 'him', 'her', 'with', 'the', 'necessary', 'skills', 'to', 'be_able', 'to', 'read', 'write', 'speak', 'and', 'understand', 'basic', 'swahili', 'swahili', 'is', 'the', 'national', 'language', 'of', 'kenya', 'and', 'tanzania', 'and', 'uganda', 'to', 'lesser', 'extent', 'it_is', 'the', 'first', 'second', 'or', 'main', 'language', 'of', 'most', 'people', 'in', 'east', 'central', 'and', 'southern', 'africa'

In [20]:
bigram[data_words[0]]

['african',
 'middle',
 'eastern',
 'languages',
 'literatures',
 'the',
 'stanford',
 'language',
 'center',
 'stanford',
 'university',
 'amelang',
 'beginning',
 'swahili',
 'course',
 'outline',
 'fall',
 'instructor',
 'sangai',
 'mohochi',
 'office_building',
 'room',
 'phone',
 'fax',
 'office_hours',
 'mt',
 'pm',
 'also',
 'by_appointment',
 'mail',
 'mohochi',
 'stanford_edu',
 'introduction',
 'this',
 'is',
 'the',
 'first',
 'part',
 'of',
 'three',
 'quarter',
 'beginning',
 'swahili',
 'course',
 'the',
 'purpose',
 'of',
 'the',
 'course',
 'is',
 'to',
 'introduce',
 'the',
 'student',
 'to',
 'swahili',
 'language',
 'and',
 'equip',
 'him',
 'her',
 'with',
 'the',
 'necessary',
 'skills',
 'to',
 'be_able',
 'to',
 'read',
 'write',
 'speak',
 'and',
 'understand',
 'basic',
 'swahili',
 'swahili',
 'is',
 'the',
 'national',
 'language',
 'of',
 'kenya',
 'and',
 'tanzania',
 'and',
 'uganda',
 'to',
 'lesser',
 'extent',
 'it_is',
 'the',
 'first',
 'second',
 'or

In [21]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [24]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['african', 'middle', 'begin', 'course', 'outline', 'fall', 'instructor', 'phone', 'fax', 'also', 'appointment', 'mail', 'introduction', 'first', 'part', 'quarter', 'begin', 'swahili', 'course', 'purpose', 'course', 'introduce', 'student', 'language', 'necessary', 'skill', 'able', 'read', 'write', 'speak', 'understand', 'basic', 'less', 'extent', 'first', 'second', 'main', 'language', 'people', 'also', 'widely', 'teach', 'broadcast', 'use', 'sub', 'saharan', 'african', 'language', 'enjoy', 'special', 'popularity', 'african', 'adopt', 'language', 'heritage', 'increasingly', 'learn', 'language', 'also', 'useful', 'research', 'professional', 'business', 'tourist', 'interest', 'addition', 'grow', 'population', 'speak', 'language', 'teach', 'college', 'high', 'elementary', 'school', 'several', 'african', 'american', 'course', 'introduction', 'language', 'culture', 'focus', 'acquire', 'basic', 'conversational', 'activity', 'enhance', 'communicative', 'skill', 'speak', 'write', 'listening', 

In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 5), (13, 5), (14, 2), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 3), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 5), (30, 1), (31, 1), (32, 1), (33, 3), (34, 1), (35, 1), (36, 1), (37, 5), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 11), (52, 1), (53, 2), (54, 1), (55, 1), (56, 1), (57, 2), (58, 3), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 8), (77, 2), (78, 6), (79, 1), (80, 1), (81, 2), (82, 5), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 1), (89, 3), (90, 1), (91, 2), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 7), (106, 1), (107, 3), (108, 2), (109, 1), (110, 1

In [27]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('absence', 2),
  ('academic_integrity', 1),
  ('accommodation', 1),
  ('acquire', 1),
  ('acquisition', 1),
  ('activity', 3),
  ('actual', 1),
  ('addition', 1),
  ('adjective', 2),
  ('administer', 1),
  ('adopt', 1),
  ('african', 5),
  ('also', 5),
  ('always_helpful', 2),
  ('american', 1),
  ('amount', 1),
  ('ancillary', 1),
  ('announce', 1),
  ('appointment', 1),
  ('appreciate', 1),
  ('appreciation', 1),
  ('approach', 1),
  ('appropriate', 3),
  ('appropriate_accommodation', 1),
  ('aramati', 1),
  ('arrange', 1),
  ('ask', 1),
  ('assessment', 1),
  ('assignment', 5),
  ('assignments_relate', 1),
  ('assistance', 1),
  ('attend', 1),
  ('attendance', 3),
  ('auxiliary_aid', 1),
  ('award', 1),
  ('base', 1),
  ('basic', 5),
  ('bear', 1),
  ('begin', 2),
  ('bind', 1),
  ('broadcast', 1),
  ('business', 1),
  ('ca', 1),
  ('cardinal', 1),
  ('ccat', 1),
  ('center', 1),
  ('chapter', 1),
  ('choice', 1),
  ('circumstance', 2),
  ('civil', 1),
  ('class', 

In [28]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [29]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.028*"arabic" + 0.012*"water" + 0.011*"week" + 0.010*"stanford_edu" + '
  '0.010*"http_www" + 0.009*"watershed" + 0.008*"model" + 0.008*"wetland" + '
  '0.007*"com" + 0.007*"lesson"'),
 (1,
  '0.033*"class" + 0.017*"student" + 0.016*"exam" + 0.015*"quiz" + '
  '0.014*"write" + 0.013*"course" + 0.012*"due" + 0.010*"instructor" + '
  '0.010*"lesson" + 0.009*"assignment"'),
 (2,
  '0.020*"class" + 0.020*"student" + 0.010*"course" + 0.010*"language" + '
  '0.009*"cultural" + 0.009*"activity" + 0.009*"african" + 0.009*"lesson" + '
  '0.009*"skill" + 0.008*"material"'),
 (3,
  '0.027*"class" + 0.026*"student" + 0.020*"unit" + 0.017*"course" + '
  '0.013*"quiz" + 0.011*"lesson" + 0.008*"oral" + 0.008*"grade" + '
  '0.007*"request" + 0.007*"bring"'),
 (4,
  '0.014*"class" + 0.013*"exercise" + 0.011*"grade" + 0.011*"course" + '
  '0.009*"unit" + 0.009*"lecture" + 0.008*"assignment" + 0.008*"include" + '
  '0.007*"water" + 0.007*"chemical"')]


In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.504137001138136

Coherence Score:  0.35428391179478264


In [31]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\simon\Anaconda3\Lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.142256 -0.046702       1        1  39.306305
3     -0.115888  0.014249       2        1  32.311012
0      0.179964 -0.110774       3        1  17.843542
4      0.091997  0.159923       4        1   5.924373
2     -0.013816 -0.016696       5        1   4.614773, topic_info=     Category        Freq           Term       Total  loglift  logprob
476   Default   85.000000         arabic   85.000000  30.0000  30.0000
51    Default  395.000000          class  395.000000  29.0000  29.0000
282   Default  281.000000        student  281.000000  28.0000  28.0000
630   Default  126.000000           unit  126.000000  27.0000  27.0000
759   Default   53.000000       exercise   53.000000  26.0000  26.0000
1326  Default   45.000000          water   45.000000  25.0000  25.0000
76    Default  207.000000         course  207.000000  24.0000  24.0000
163   Default   95.000000       language   95.000000  23.0000  23.0000
6     Default   37.000000       activity   37.000000  22.0000  22.0000
124   Default  101.000000          grade  101.000000  21.0000  21.0000
105   Default  121.000000           exam  121.000000  20.0000  20.0000
136   Default   39.000000       http_www   39.000000  19.0000  19.0000
1327  Default   28.000000      watershed   28.000000  18.0000  18.0000
298   Default   84.000000           text   84.000000  17.0000  17.0000
272   Default   32.000000          skill   32.000000  16.0000  16.0000
177   Default   42.000000       material   42.000000  15.0000  15.0000
839   Default   37.000000         follow   37.000000  14.0000  14.0000
246   Default  169.000000           quiz  169.000000  13.0000  13.0000
403   Default  152.000000         lesson  152.000000  12.0000  12.0000
316   Default   99.000000           week   99.000000  11.0000  11.0000
141   Default   28.000000        include   28.000000  10.0000  10.0000
1192  Default   26.000000          model   26.000000   9.0000   9.0000
255   Default   70.000000        require   70.000000   8.0000   8.0000
1331  Default   23.000000        wetland   23.000000   7.0000   7.0000
78    Default   15.000000       cultural   15.000000   6.0000   6.0000
1608  Default   36.000000  pronunciation   36.000000   5.0000   5.0000
77    Default   25.000000          cover   25.000000   4.0000   4.0000
490   Default   24.000000            com   24.000000   3.0000   3.0000
148   Default  115.000000     instructor  115.000000   2.0000   2.0000
300   Default   58.000000          topic   58.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
314    Topic5    3.263504            web    8.257174   2.1476  -5.4482
237    Topic5    3.234071        primary    8.159618   2.1505  -5.4573
267    Topic5    2.218655          shall    4.784988   2.3073  -5.8341
210    Topic5    2.041996        outline    4.258902   2.3408  -5.9171
272    Topic5    6.692921          skill   32.600399   1.4926  -4.7300
6      Topic5    6.831268       activity   37.254898   1.3796  -4.7095
77     Topic5    4.878146          cover   25.889145   1.4068  -5.0463
177    Topic5    6.062793       material   42.089169   1.1383  -4.8288
9      Topic5    2.332214      adjective    6.522256   2.0475  -5.7842
282    Topic5   15.115001        student  281.037323   0.1531  -3.9153
51     Topic5   15.206025          class  395.801392  -0.1833  -3.9093
163    Topic5    7.378462       language   95.602798   0.5143  -4.6325
126    Topic5    3.069306       greeting   12.577255   1.6655  -5.5096
298    Topic5    5.771699           text   84.018494   0.3978  -4.8781
466    Topic5    2.797775        various   12.390760   1.5878  -5.6022
76     Topic5    7.835208         course  207.370026  -0.2000  -4.5724
403    Topic5    6.786741         lesson  152.751511  -0.0379  -4.7160
141    Topic5    3.669397        include   28.990519   1.0090  -5.3310
296    Topic5    4.761478    